In [ ]:
#Run the kSZ 4-point function
#
#
from os.path import join as opj, dirname
import os
os.environ["DISABLE_MPI"]="true"
#from ksz4.cross import four_split_K, split_phi_to_cl
#from ksz4.reconstruction import setup_recon, setup_ABCD_recon, get_cl_smooth
from pixell import curvedsky, enmap
from scipy.signal import savgol_filter
from ksz4.utils import ClBinner
from falafel import utils, qe
import healpy as hp
import yaml
import argparse
from orphics import maps
import numpy as np
import matplotlib.pyplot as plt
import pickle
from plot_tools import *

In [ ]:
log_binner=False
binner = ClBinner(lmin=5, lmax=300, nbin=8, log=log_binner)

In [ ]:
%matplotlib inline

mask_60sk_file = "/global/cfs/projectdirs/act/data/synced_maps/DR6_lensing/masks/act_mask_20220316_GAL060_rms_60.00sk.fits"

fig,ax=plt.subplots(figsize=(5,5))
#fig_n0, ax_n0 = plt.subplots(figsize=(5,4))
#lpower=2
#plot_clKK(ax, "90_90_90_90", binner, color='C0', label="90 only", lfac=lfac)
raw_auto=False
yscale="log"
linthreshy=1.
offsets=np.linspace(-0.05,0.05,6)
plot_rdn0=False
plot_meanfield_correction=False
lpower=2
plot_raw_auto=False
plot_mcn0=False
do_one_leg_cleaned=False
plot_meanfield=False
ylim=None #[-0.5,2]
divide_by_uL=False
use_alt_norm=False
use_r24_norm=True
plot_theory=True
plot_scaled_theory=False
transfer_func=0.94
check_auto=False

d4000 = "/pscratch/sd/m/maccrann/ksz_outputs/output_hilc_hilc_hilc_hilc_dr6v4_v4_lmax4000_mask60sk_noisysims_re-run"
clkk_stuff_4000 = plot_clKK(ax, d4000, 
           binner, mask_60sk_file, color='C0', rdn0_file="rdn0_outputs_nsim64.pkl",
           label=r"ACT DR6 + $\it{Planck}$",  
           yscale=yscale,  lpower=lpower,
           nsim_meanfield=64, plot_rdn0=plot_rdn0, plot_mcn0=plot_mcn0,
           add_offset = 0., est="qe", divide_by_uL=divide_by_uL, check_auto=check_auto,
           add_to_err=0., #binner(fg_terms_iici["trispectrum_qe"]), 
           plot_meanfield=plot_meanfield, plot_raw_auto=plot_raw_auto,
           plot_meanfield_correction=plot_meanfield_correction, plot_theory=plot_theory,
         sim_CLKKs_file="/pscratch/sd/m/maccrann/ksz_outputs/sim_e2e_test/output_auto_config_lmax4000_hilc_hilc_hilc_hilc/output_auto_config_lmax4000_hilc_hilc_hilc_hilc_CLKKs_nsim128.npy",
         plot_scaled_theory=plot_scaled_theory,
         transfer_func=0.94, use_alt_norm=use_alt_norm, use_r24_norm=use_r24_norm)

if (use_alt_norm or use_r24_norm):
    print(clkk_stuff_4000["alt_norm_over_orig_norm"])
    fg_prefactor=clkk_stuff_4000["alt_norm_over_orig_norm"]
else:
    fg_prefactor=None
"""
#Get foreground contribution estimated from websky
fg_bias_websky_iiii_lmax4000,fg_prof = get_fg_trispectrum("allfgs_nemo-wdr6dn_tsz-nemo-model-snr5_ps-model-snr4_wplanck-ps2_mask60sk_dataCl",
                                         "lmin3000_lmax4000",
                                         "ilc_ilc_ilc_ilc", "qe",
                                        divide_by_uL=divide_by_uL, lpower=lpower, 
                                                    prefactor=fg_prefactor)
assert np.allclose(fg_prof[:4000], clkk_stuff_4000["profile"][:4000])
ax.plot(binner.bin_mids, 
        binner(fg_bias_websky_iiii_lmax4000[:binner.lmax+1]), '--', color="C0")
ax.plot([],[],'--',color='C0', label="websky")

fg_bias_sehgal_iiii_lmax4000,_ = get_fg_trispectrum("sehgal_allfgs_nemo-wdr6dn_tsz-nemo-model-snr5_ps-model-snr4_wplanck-ps2-no545_mask60sk_dataCl",
                                         "lmin3000_lmax4000",
                                         "ilc_ilc_ilc_ilc", "qe",
                                                 divide_by_uL=divide_by_uL, lpower=lpower,
                                                   prefactor=fg_prefactor)
ax.plot([],[],':',color='C0', label="S10")
ax.plot(binner.bin_mids, 
        binner(fg_bias_sehgal_iiii_lmax4000[:binner.lmax+1]), ':', color='C0')
"""

if plot_meanfield:
    ax.plot([],[],":",color="k", label="meanfield auto")
if plot_rdn0:
    ax.plot([],[],"--", color="k", label="RDN0")
if plot_meanfield_correction:
    ax.plot([],[],linestyle=(0, (3, 1, 1, 1)), color='k', label="meanfield correction")
if plot_mcn0:
    ax.plot([],[],linestyle="-.", color='k', label="MCN0")
if plot_raw_auto:
    ax.errorbar([],[],yerr=[],fmt='o',color='k',mfc='none',label="raw auto")
if plot_theory:
    ax.plot([],[],"-",color="k", label="Alvarez 2016")
    

lg=ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.13), ncol=2, reverse=True)

#if binner.log:
#ax.set_xscale('log')
if yscale=="symlog":
    ax.set_yscale(yscale, linthreshy=linthreshy)
else:
    ax.set_yscale(yscale)
ax.set_xlabel(r"$L$")
if lpower!=0:
    ax.set_ylabel(r"$L^{%d}C_L^{KK}$"%lpower)
else:
    ax.set_ylabel(r"$C_L^{KK}$")
    
if ylim is not None:
    ax.set_ylim(ylim)
    
fig.tight_layout()
plt.subplots_adjust(bottom=0.28)

new_plot_dir="re-run_plots"
os.makedirs(new_plot_dir, exist_ok=True)
filename=opj(new_plot_dir, "CLKK_dr6_baseline_hilc_v4_re-run.png")
if use_alt_norm:
    filename=opj(new_plot_dir, "CLKK_dr6_baseline_hilc_v4_alt_norm_re-run.png")
elif use_r24_norm:
    filename=opj(new_plot_dir, "CLKK_dr6_baseline_hilc_v4_r24_norm_re-run.png")
fig.savefig(filename, dpi=300, bbox_extra_artists=(lg,))